In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
sns.set(
    font_scale=2,
    style="whitegrid",
    rc={'figure.figsize':(20,7)}
        )

In [ ]:
clients = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-bundych/shared/homeworks/python_ds_miniprojects/6/ads_clients_data.csv', parse_dates=['date', 'create_date'])
data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-bundych/shared/homeworks/python_ds_miniprojects/6/ads_data.csv', parse_dates=['date'])

In [ ]:
data.head()

In [ ]:
clients.head()

In [ ]:
#посчитать кол-во событий по каждому объявлению

count_by_event = data.groupby(by=['ad_id', 'event']).agg({'date': 'count'}).reset_index()
count_by_event.head()

In [ ]:
#сводная таблица по событиям объявлений

pivot_by_event = pd.pivot_table(count_by_event, index='ad_id' , columns='event', values='date')
pivot_by_event = pivot_by_event.fillna(0) #заполнить пустые значения, чтобы рассчет среднего был корректным

pivot_by_event.head()

In [ ]:
#среднее число событий на одно объявление

view_mean = round(pivot_by_event.view.mean())
click_mean = round(pivot_by_event.click.mean())

print("view_mean: ", view_mean)
print("click_mean: ", click_mean)

In [ ]:
#распределение показов

pivot_by_event_no_0 = pivot_by_event.query('view != 0') #убираем нули из распределения чтобы не было бесконечности при 
                                                        #вычислении логарифма от 0
sns.distplot(np.log(pivot_by_event_no_0['view'])) #логарифм для адекватной картинки распределения

In [ ]:
##среднее кол-во показов объявлений по дням

#кол-во показов в день по объявлениям
views_by_day = data.query('event == "view"') \
                .groupby(['date', 'ad_id']) \
                .agg({'time': 'count'})
views_by_day.head()

In [ ]:
#средее кол-во показов в день

views_by_day_mean = views_by_day.groupby('date').agg({'time': 'mean'}).reset_index()
views_by_day_mean

In [ ]:
#скользящее среднее по дням (окно=2)

views_by_day_mean['ma'] = views_by_day_mean['time'].rolling(2).mean()
views_by_day_mean = views_by_day_mean.rename(columns={'time': 'mean'})
views_by_day_mean

In [ ]:
# показать на одном графике значения среднего и скользящего среднего кол-ва показов
sns.lineplot(data=views_by_day_mean[['mean','ma']])

#наибольшая разница по модулю м\у средним и скольз.средним
views_by_day_mean['diff'] = abs(views_by_day_mean['mean'] - views_by_day_mean['ma'])
views_by_day_mean

In [ ]:
#объединить 2 ДФ

full_data = data.merge(clients, on='client_union_id')

In [ ]:
#создаем колонку с разницей дата события от даты создания
full_data['create_diff'] = (full_data['create_date'] - full_data['date_x']) * (-1) #перевод к положит числу

#берем минимальное значение разницы(т.е. разницу на момент наступления перового события) по каждому клиенту
client_min_dif = full_data.groupby('client_union_id').agg({'create_diff': 'min'}).reset_index()
print(client_min_dif.head())

#среднее значение разницы от регистрации до запуска рекламы.
client_min_dif['create_diff'].mean()

In [ ]:
#конверсия из создания кабинета в запуск рекламы в течение 365 дней.

clients_365 = client_min_dif.query('create_diff <= @pd.Timedelta(365, unit="day")')
clients_365_count = clients_365.create_diff.count()
clients_total = clients.client_union_id.nunique()

print('кол-во клиентов, запустивших рекламу:', clients_365_count)
print('кол-во клиентов всего:', clients_total)
print('конверсия кабинет=реклама:', round(clients_365_count / clients_total * 100, 2))


In [ ]:
#кол-во клиентов, которые запустили рекламу в течение 30,90,180,365 дней

client_min_dif['group'] = pd.cut(client_min_dif.create_diff, pd.to_timedelta(['0d', '30d', '90d', '180d', '365d']))
users_count_intervals = client_min_dif.groupby('group', as_index=False).agg({'client_union_id': 'count'})
users_count_intervals['days'] = ['30 days', '90 days','180 days', '365 days']

users_count_intervals

In [ ]:
import plotly.express as px

fig = px.bar(users_count_intervals, x='days', y='client_union_id')
fig.show()